## 通过Segmentator对CT，进行lung区域的自动化分割，从而帮助提取ROI

In [ ]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
import scipy.ndimage as ndi

from totalsegmentator.python_api import totalsegmentator

def lung_segment():
    data_root = '/mnt/HDD_1/FDG/LungCancer_Subtyping/data/nifti'
    out_root = 'data/lung-seg'
    os.makedirs(out_root, exist_ok=True)

    locations = ['AKH_nifti_637', 'Neimeng_nifti_425']
    for loca in locations:
        df = pd.read_excel(f'data/{loca}_image_info.xlsx', sheet_name='Sheet1', dtype={'PID': str})
        patient_list = df['PID'].values
        for pid in patient_list:
            CT_path = os.path.join(data_root, loca, pid, 'CT.nii.gz')
            output_path = os.path.join(out_root, loca, pid)

            if not os.path.exists(CT_path):
                print(f"Missing CT for patient: {pid}")
                continue

            try:
                print(f"[INFO] Preprocessing {loca} {pid}")
                totalsegmentator(
                    input=CT_path,
                    output=output_path,
                    task="lung_nodules"
                )
            except Exception as e:
                print(f"Error processing {loca} {pid} - {str(e)}")
                with open("seg_error_log.txt", "a") as f:
                    f.write(f"{loca} {pid} - {str(e)}\n")


if __name__ == '__main__':
    lung_segment()

### 在原始CT的数据上，产生的分割后的lung mask，移动到 /mnt/HDD_1/FDG/LungCancer_Subtyping/data/lung_seg 文件夹下得以保存

In [2]:
# 移动lung-seg文件夹到目标位置
import os
import shutil
from pathlib import Path

# 源文件夹路径
source_dir = '/home/yaobo/Project/LungCancer/data/lung-seg'
# 目标文件夹路径
target_dir = '/mnt/HDD_1/FDG/LungCancer_Subtyping/data/lung_seg'

print(f"源文件夹: {source_dir}")
print(f"目标文件夹: {target_dir}")

# 检查源文件夹是否存在
if os.path.exists(source_dir):
    print(f"✓ 源文件夹存在")
    
    # 检查源文件夹内容
    source_path = Path(source_dir)
    if source_path.is_dir():
        contents = list(source_path.iterdir())
        print(f"源文件夹包含 {len(contents)} 个项目:")
        for item in contents[:10]:  # 显示前10个项目
            print(f"  - {item.name}")
        if len(contents) > 10:
            print(f"  ... 还有 {len(contents) - 10} 个项目")
    
    # 创建目标文件夹的父目录
    target_parent = Path(target_dir).parent
    target_parent.mkdir(parents=True, exist_ok=True)
    print(f"✓ 目标父目录已准备: {target_parent}")
    
    # 检查目标文件夹是否已存在
    if os.path.exists(target_dir):
        print(f"⚠ 目标文件夹已存在: {target_dir}")
        print("请选择操作:")
        print("1. 删除现有目标文件夹并移动")
        print("2. 合并内容到现有目标文件夹")
        print("3. 取消操作")
        
        # 这里我们选择合并内容的方式
        choice = "2"  # 可以根据需要修改
        
        if choice == "1":
            shutil.rmtree(target_dir)
            print(f"✓ 已删除现有目标文件夹")
            shutil.move(source_dir, target_dir)
            print(f"✓ 文件夹移动完成: {source_dir} -> {target_dir}")
        elif choice == "2":
            # 合并内容
            print(f"开始合并内容...")
            source_path = Path(source_dir)
            target_path = Path(target_dir)
            
            for item in source_path.iterdir():
                dest_item = target_path / item.name
                if item.is_dir():
                    if dest_item.exists():
                        print(f"  合并目录: {item.name}")
                        shutil.copytree(item, dest_item, dirs_exist_ok=True)
                    else:
                        print(f"  移动目录: {item.name}")
                        shutil.move(str(item), str(dest_item))
                else:
                    print(f"  移动文件: {item.name}")
                    shutil.move(str(item), str(dest_item))
            
            # 删除空的源文件夹
            if not any(source_path.iterdir()):
                source_path.rmdir()
                print(f"✓ 已删除空的源文件夹")
            
            print(f"✓ 内容合并完成")
        else:
            print("操作已取消")
    else:
        # 直接移动
        shutil.move(source_dir, target_dir)
        print(f"✓ 文件夹移动完成: {source_dir} -> {target_dir}")
    
    # 验证移动结果
    if os.path.exists(target_dir):
        target_path = Path(target_dir)
        final_contents = list(target_path.iterdir())
        print(f"\n✓ 移动完成! 目标文件夹现在包含 {len(final_contents)} 个项目:")
        for item in final_contents[:10]:
            print(f"  - {item.name}")
        if len(final_contents) > 10:
            print(f"  ... 还有 {len(final_contents) - 10} 个项目")
    else:
        print("❌ 移动失败，目标文件夹不存在")

else:
    print(f"❌ 源文件夹不存在: {source_dir}")
    print("请检查路径是否正确")

源文件夹: /home/yaobo/Project/LungCancer/data/lung-seg
目标文件夹: /mnt/HDD_1/FDG/LungCancer_Subtyping/data/lung_seg
✓ 源文件夹存在
源文件夹包含 2 个项目:
  - Neimeng_nifti_425
  - AKH_nifti_637
✓ 目标父目录已准备: /mnt/HDD_1/FDG/LungCancer_Subtyping/data
⚠ 目标文件夹已存在: /mnt/HDD_1/FDG/LungCancer_Subtyping/data/lung_seg
请选择操作:
1. 删除现有目标文件夹并移动
2. 合并内容到现有目标文件夹
3. 取消操作
开始合并内容...
  移动目录: Neimeng_nifti_425
  移动目录: AKH_nifti_637
  移动目录: AKH_nifti_637
✓ 已删除空的源文件夹
✓ 内容合并完成

✓ 移动完成! 目标文件夹现在包含 2 个项目:
  - AKH_nifti_637
  - Neimeng_nifti_425
✓ 已删除空的源文件夹
✓ 内容合并完成

✓ 移动完成! 目标文件夹现在包含 2 个项目:
  - AKH_nifti_637
  - Neimeng_nifti_425
